In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "6"
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"

In [3]:
import pandas as pd
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import ModelCheckpoint
from sklearn.model_selection import train_test_split
from transformers import T5Tokenizer

In [4]:
import random
import numpy as np
import torch
import re

def seed_everything(seed=10):
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True


seed_everything()

### Данные

In [5]:
# [pin]
file_path = "../data/data-hard2.csv"
root_path = "../data/"

df = pd.read_csv(file_path)
df["prefix"] = "clsorg"
df = df.rename({"message": "input_text", "label": "target_text"}, axis=1)

# df["input_text"] = df["input_text"].apply(preprocess_text)
df.drop(2764)
df.sample(20)

,input_text,target_text,prefix
3538,​​YouTube хотят заблокировать переведя всех в ...,223-4,clsorg
7019,🧑‍💻 Софтлайн запускает программу мотивации сот...,237-4,clsorg
4123,⚠️🇷🇺#PLZL #дивиденд сд Полюс: не выплачивать ...,127-2,clsorg
6912,🟢 Полюс — самая надежная инвестиция в российск...,127-4;127-5,clsorg
3164,Совет директоров Полюса рекомендовал не выплач...,127-2,clsorg
4644,🇷🇺#FEES #цифровизация «Россети» создадут цифр...,186-3,clsorg
5796,💥🇷🇺#MTLR = +5%,99-3,clsorg
3315,Чистая прибыль банковского сектора в марте 202...,150-4,clsorg
824,"""🇷🇺#SMLT #отчетность """"Самолет"""" в IV кв. уве...",56-4,clsorg
610,"""🇷🇺#BELU """"Белуга Групп"""" сменила название на ...",36-3,clsorg


### Modeling

In [6]:
m_name = "cointegrated/rut5-small"
tokenizer = T5Tokenizer.from_pretrained(m_name)

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [7]:
import sys

sys.path.append("/home/worker/workspace/hakatons/hakaton-gagarin-sentiment_interface/src")

from t5.dataset import NERDataModel
from t5.model import NERModel

BATCH_SIZE = 128
EPOCHS = 5
train_df, test_df = train_test_split(df, test_size=0.25, random_state=42)
data_module = NERDataModel(train_df, test_df, tokenizer, batch_size=BATCH_SIZE)
data_module.setup()

In [ ]:
model = NERModel(m_name, lr=0.0006)

checkpoint_callback = ModelCheckpoint(
    dirpath="checkpoints2",
    filename="ner",
    save_top_k=1,
    verbose=True,
    monitor="val_loss",
    mode="min",
)

trainer = Trainer(
    callbacks=[checkpoint_callback],
    max_epochs=EPOCHS,
    accelerator="cuda",
)

In [11]:
!rm -r lightning_logs

In [12]:
trainer.fit(model, data_module)

You are using a CUDA device ('NVIDIA A100 80GB PCIe') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
Missing logger folder: /home/worker/workspace/hakatons/hakaton-gagarin-sentiment_interface/pybooks/lightning_logs
/home/worker/workspace/hakatons/hakaton-gagarin-sentiment_interface/.conda/lib/python3.10/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:653: Checkpoint directory /home/worker/workspace/hakatons/hakaton-gagarin-sentiment_interface/pybooks/checkpoints2 exists and is not empty.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [6]
/home/worker/workspace/hakatons/hakaton-gagarin-sentiment_interface/.conda/lib/python3.10/site-packages/transformers/optimization.py:429: FutureWarning: This implementation of AdamW is depre

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/home/worker/workspace/hakatons/hakaton-gagarin-sentiment_interface/.conda/lib/python3.10/site-packages/pytorch_lightning/loops/fit_loop.py:298: The number of training batches (42) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.


Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Epoch 2, global step 126: 'val_loss' reached 0.93560 (best 0.93560), saving model to '/home/worker/workspace/hakatons/hakaton-gagarin-sentiment_interface/pybooks/checkpoints2/ner-v25.ckpt' as top 1


Validation: |          | 0/? [00:00<?, ?it/s]

Epoch 3, global step 168: 'val_loss' reached 0.91776 (best 0.91776), saving model to '/home/worker/workspace/hakatons/hakaton-gagarin-sentiment_interface/pybooks/checkpoints2/ner-v25.ckpt' as top 1


Validation: |          | 0/? [00:00<?, ?it/s]

Epoch 4, global step 210: 'val_loss' was not in top 1
`Trainer.fit` stopped: `max_epochs=5` reached.


In [ ]:
# model.model.save_pretrained('models/pretrained-rut5-2')

### Test model

In [8]:
import torch

m_name = 'models/pretrained-rut5-2'
model = NERModel(
    m_name
)

In [9]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)

NERModel(
  (model): T5ForConditionalGeneration(
    (shared): Embedding(20100, 512)
    (encoder): T5Stack(
      (embed_tokens): Embedding(20100, 512)
      (block): ModuleList(
        (0): T5Block(
          (layer): ModuleList(
            (0): T5LayerSelfAttention(
              (SelfAttention): T5Attention(
                (q): Linear(in_features=512, out_features=384, bias=False)
                (k): Linear(in_features=512, out_features=384, bias=False)
                (v): Linear(in_features=512, out_features=384, bias=False)
                (o): Linear(in_features=384, out_features=512, bias=False)
                (relative_attention_bias): Embedding(32, 6)
              )
              (layer_norm): T5LayerNorm()
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (1): T5LayerFF(
              (DenseReluDense): T5DenseGatedActDense(
                (wi_0): Linear(in_features=512, out_features=1024, bias=False)
                (wi_1): Linear(in_fe

In [10]:
from t5.utils import evaluate_metric, generate_answer_batched

with torch.inference_mode():
    predictions = generate_answer_batched(
        trained_model=model,
        tokenizer=tokenizer,
        data=test_df[:1200],
        batch_size=64,
    )


100%|██████████| 19/19 [00:16<00:00,  1.17it/s]


In [11]:
ldf = test_df[:1200].copy()
ldf["predictions"] = predictions

In [14]:
ldf

,input_text,target_text,prefix,predictions,tcomp,tsent,pcomp,psent
3518,​Яндекс разбил мне сердце и я его продал 💔 Я ...,236-2,clsorg,236-3,236,2,236,3
319,"""Совет директоров ПАО """"Новолипецкий металлург...",116-2,clsorg,116-3,116,2,116,3
625,"""🇷🇺#CHMF #мнение """"Северсталь"""" в 2023 году ...",152-4,clsorg,152-4,152,4,152,4
3053,СД Северстали рекомендовал акционерам одобрить...,152-4,clsorg,152-4,152,4,152,4
2328,Главное к открытию среды (21.06): #брифинг 📌 ...,25-3,clsorg,251-5,25,3,251,5
...,...,...,...,...,...,...,...,...
805,"""🇷🇺#SBER #event Совфед пригласил Грефа выступ...",150-3,clsorg,150-4,150,3,150,4
410,"""⏰ 23 января - Доброе утро! 🇧🇷🇦🇷 Бразилия и А...",53-4,clsorg,53-2,53,4,53,2
3821,​​🟢 ИТОГИ ДНЯ. Российские акции дорожают ⚪️ ...,160-4;4-2,clsorg,160-4,160,4,160,4
3570,​​Глянул отчёт по сегеже за 22 год Смогли до...,204-3,clsorg,152-4,204,3,152,4


In [15]:
def format_predictions(predictions):
    predictions = re.sub(r'[^0-9;-]', '', predictions)
    pattern = r'\d+-\d+'
    matches = re.findall(pattern, predictions)
    if not matches:
        matches = ["150-3"]
    return ';'.join(matches)

ldf['predictions'] = ldf['predictions'].apply(format_predictions)

In [16]:
ldf[["tcomp", "tsent"]] = (
    ldf["target_text"].str.split(";", expand=True)[0].str.split("-", expand=True)
)
ldf[["pcomp", "psent"]] = (
    ldf["predictions"].str.split(";", expand=True)[0].str.split("-", expand=True)
)

In [17]:
evaluate_metric(
    company_predictions=ldf["pcomp"].tolist(),
    company_labels=ldf["tcomp"].tolist(),
    sentiment_predictions=ldf["psent"].tolist(),
    sentiment_labels=ldf["tsent"].tolist(),
)

{'total': 66.74816859422192, 'f1': 0.7199633718844386, 'accuracy': 0.615}